# DEMO: EUCLID-BOX

**Euclid-box**: this repository contains the computation of the theoretical predictions for Euclid observables as well as the computation of the likelihood given some fiducial data. The likelihood is designed to work as an external likelihood for the Bayesian Analysis Code `Cobaya`.

**Disclaimer**: `Cobaya`
 can be used for many purposes
* To run Bayesian statistical analysis using the external EuclidLikelihood class within the Euclid-box
* To compute the theoretical requirements of the Euclid-box (matter power spectrum, angular and comoving distances, H(z)...) by choosing from different Boltzmann Solvers (i.e: CAMB/CLASS)
* it offers several samplers to perform the Bayesian statistical analysis (MCMC, Polychord, minimizer...)


**Install**: To start playing with this notebook, install the ```Likelihood Package``` and its modules according to README.
Alternatively you may be ready to run if ```Cobaya``` and ```CAMB/CLASS``` are installed and the `modules_path` variable points to your `Cobaya` modules

**Euclid-box documentation**: http://pf-ist-likelihood.pages.euclid-sgs.uk/likelihood-implementation/index.html

**Cobaya documentation**: https://cobaya.readthedocs.io/en/latest/

**Python information**: if the user is not confortable with some of the python vocabulary used in this notebook,
ne can find a nice python review here
https://wiki.python.org/moin/BeginnersGuide

In [ ]:
# General python imports

import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy import interpolate 
import time 

In [ ]:
# Matplotlib params set-up

%matplotlib inline
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('font',size=25)
plt.rc('axes', titlesize=26)
plt.rc('axes', labelsize=25)
plt.rc('lines', linewidth=2)
plt.rc('lines', markersize=6)
plt.rc('legend', fontsize=20)
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='STIXGeneral')

In [ ]:
# Cobaya needs some modules to run
# i.e: CAMB, CLASS, Polychord, other likelihood codes and data (Planck 18, DES...)
# If you have installed the Euclid-box as in the README instructions, you just need to activate 
# the conda environment `likelihood` to run 
# This conda environment has installed CAMB so you won't need to worry about anything else

# Alternatively, if you have already installed Cobaya and other Cosmological codes 
# (CAMB, CLASS, Polychord, Planck 18) you can 
# (1): simply point out where each of them is installed with the flag 'path' in the dictionary below
# (2): if you installed the Cosmological codes as Cobaya automatic installation suggest
# you need to point out the path to your modules
# (https://cobaya.readthedocs.io/en/latest/installation_cosmo.html)

modules_path = "/data2/cobaya_modules/"

## CASE 1: 'Run `Cobaya` with Euclid-Likelihood and `evaluate` sampler (or any other sampler within `Cobaya`)

**User-case**: *Run and go*. This is the most straightforward case where the user will simply call Cobaya to make an evaluation of the likelihood given some theoretical predictions of Euclid observables

In [ ]:
# Import external loglike from the Likelihood Package within cobaya_interface.py

from likelihood.cobaya_interface import EuclidLikelihood

In [ ]:
# We are running the Euclid-Likelihood as an external likelihood class for Cobaya
# Cobaya needs a dictionary or yaml file as input to start running

info = {
    #Which parameters would you like to sample?
    'params': {
        # If the prior dictionary is not passed to a parameter, this parameter is fixed
        'ombh2': 0.022445, 'omch2': 0.1205579307, 'H0': 67, 'tau': 0.0925,
        'mnu': 0.06, 'nnu': 3.046, 'As': 2.12605e-9,
        'ns': 0.96, 
        # Which observational probe would you like to use?
        # 1: photo-z
        # 2: spec
        # 12: both
        'like_selection': 2,
        # Want to use GC+XC+WL in photo? Then full_phot must be True
        'full_photo': False,
        # Bias parameters
        # Photo IST:F values
        'b1_photo': 1.0997727037892875,
        'b2_photo': 1.220245876862528,
        'b3_photo': 1.2723993083933989,
        'b4_photo': 1.316624471897739,
        'b5_photo': 1.35812370570578,
        'b6_photo': 1.3998214171814918,
        'b7_photo': 1.4446452851824907,
        'b8_photo': 1.4964959071110084,
        'b9_photo': 1.5652475842498528,
        'b10_photo': 1.7429859437184225,
        # Spec IST:F values
        'b1_spec': 1.46,
        'b2_spec': 1.61,
        'b3_spec': 1.75,
        'b4_spec': 1.90},
    # Which theory code you want to use, CAMB/CLASS? 
    # Here we use CAMB and specify some CAMB args
    'theory': {'camb': {'stop_at_error': True, 'extra_args':{'num_massive_neutrinos': 1}}},
    # Which sampler do you want to use? Check COBAYA docs to see the list of available samplers
    # Here we use 'evaluate'
    'sampler': {'evaluate': None},  
    # Where have you installed your COBAYA modules (i.e: CAMB, polychord, likelihoods...)?
    'packages_path': modules_path,
    # Where are the results going to be stored, in case the sampler produce output files? 
    'output': 'chains/my_euclid_experiment',
    # Likelihood: we load the Euclid-Likelihood as an external function, and name it 'Euclid'
    'likelihood': {'Euclid': EuclidLikelihood},
    # How much infomartion you want Cobaya to print? If debug = True, it prints every single detail
    'debug': False,
    # If timing = True, Cobaya returns how much time it took him to make a computation of the posterior
    'timing': True,
    # This command below forces deleting the previous output files, if found, with the same name
    'force': True
    }

In [ ]:
# Import Cobaya -need to be installed
from cobaya.run import run

# Let's run Cobaya
# the function run returns
# info_updated: an information dictionary updated with the defaults, 
# equivalent to the updated yaml file produced by the shell invocation
# samples: a sampler object, with a sampler.products() 
# being a dictionary of results. 
# For the mcmc sampler, the dictionary contains only one chain under the key sample.

info_updated, samples = run(info)

**Comment**: you see as Cobaya output:
- output_mpi: it tells you details about the output 
- Prior: which parameters are you sampling? In this case, none, this is why it reminds you that.
- CAMB: it calls the theory code you wanted to use (CAMB/CLASS, in this case, CAMB, and where it is installed)

You see these outputs repeated twice because you are internally calling `Cobaya`, within the EuclidLikelihood, to calculate the fiducial cosmology.
- evaluate: the evaluate sampler gets initialized, looks for a point and evaluates the posterior. 

Note that because the option timing is True in the info dictionary, it will tell you how much time it took to compute the likelihood (euclid) and CAMB





# Case 2: Run `Cobaya model` with Euclid-Likelihood as external likelihood 

**User-case**: imagine you are running a MCMC sampling to find the best-fit values of $\Lambda CDM$. Because of an unknown reason, your MCMC chains are getting stuck. The `wrapper model` of Cobaya allows you to investigate what's going on within Cobaya at each step of the algorithm. It's a wrapper of the `evaluate` sampler above, which also gives you the possibility of understanding, seeing and retrieving derived parameters and quantities.

In [ ]:
# Import model wrapper of Cobaya to play around
from cobaya.model import get_model

# You can reproduce the option 'evaluate' with the model wrapper of Cobaya, which gives you more flexibility
# Create a Cobaya model using the model wrapper of Cobaya. Remember, as input, it needs a yaml or dictionary
# we measure the time to gives us an estimation of how much time it takes to make the inizialitation of the 
# likelihood

t1 = time.time()
model = get_model(info)
print('Time for initialization of the likelihood: ', time.time()-t1)

**Comment**: you see as Cobaya output:
- Model: it tells you Cobaya is using the `model` wrapper and it's reading the info dictionary.
- Prior: which parameters are you sampling? In this case, none, this is why it reminds you that.
- CAMB: it calls the theory code you wanted to use (CAMB/CLASS, in this case, CAMB, and where it is installed)

You see these outputs repeated twice because you are internally calling `Cobaya`, within the EuclidLikelihood, to calculate the fiducial cosmology.

It takes around 10 seconds to initialize the likelihood (reading OU-LE3 data and compute fiducial cosmology) 

In [ ]:
# You can have an insight of which theory requirements have been requesting:
# the likelihood is asking to the theory code (aka CAMB)?
print('\n Requirements \n')
print(model.provider.requirement_providers)
# So, what have we requested?
print('\n Requested \n')
print(model.requested())

In [ ]:
# To execute the likelihood with the model wrapper, you need to make an evaluation of the posterior
# Attention: if a parameter is sampled, we need to obtain a value from the prior
# i.e: 
# point = dict(zip(model.parameterization.sampled_params(),
#                 model.prior.sample(ignore_external=True)[0]))
# You need to pass this point within the logposterior({point})
#
# If all the parameters are fixed, just call the posterior
#
# Once again, we measure the time just to have an idea of the computation time of the likelihood

t1 = time.time()
logposterior = model.logposterior({})
t2 = time.time()

print('Time to compute the logposterior: ', t2-t1)
print('Full log-posterior:')
print('   logposterior: %g' % logposterior.logpost)
print('   logpriors: %r' % dict(zip(list(model.prior), logposterior.logpriors)))
print('   loglikelihoods: %r' % dict(zip(list(model.likelihood), logposterior.loglikes)))
print('   derived params: %r' % dict(zip(list(model.parameterization.derived_params()), logposterior.derived)))

# Case 3: EuclidLikelihood package

**User-case**: imagine you have actually loaded the `model` wrapper of Cobaya, but you are unable yet to find what the issue is. In this case, we aim to go one level deeper in the computation: we are going to have a grasp of what EuclidLikelihood actually does (theoretical recipe of the observables, reading the data, computation of the $\chi^2$). 

**Disclaimer**: the steps below are done internally by `Cobaya` at each step of the sampling procedure within the file `cobaya_interface.py` of the likelihood package.

In [ ]:
# Create an instance of the class EuclidLikelihood
like = EuclidLikelihood()

# Initialize default parameters for redshift, k-array, fiducial cosmology...
like.initialize()

# Get the cosmo_dictionary where all the cosmology + theory parameters are saved
# ATTENTION: you need to pass the `cobaya wrapper` model initialized above as an argument of the function
# as well as the parameters of your theory
# When only Cobaya is used (like the user case 1), it creates internally this model instance
like.passing_requirements(model, **model.provider.params)

# Update the cosmology dictionary with interpolators + basic quantities such as
# P_gg, P_delta...
# DISCLAIMER: this cosmology dictionary cosmo_dic is an attribute of the class Cosmology but it is also
# saved as an attribute of the class EuclidLikelihood
# We also save the 
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], 0.05)


# Show what the cosmo_dic actually contains
print('\nKeys of the cosmo_dic: \n', list(like.cosmo.cosmo_dic.keys()))
print('\nKeys of the input params within cosmo_dic: \n', list(like.cosmo.cosmo_dic['nuisance_parameters'].keys()))

### Plot internal quantities and cosmological observables

In [ ]:
# Let's plot the background quantities of cosmo_dic
# See http://pf-ist-likelihood.pages.euclid-sgs.uk/likelihood-implementation/likelihood.cosmo.cosmology.html
# for extra information

fig, axs = plt.subplots(2, 2, figsize=(18,10))
fig.suptitle('Background quantities')
axs[0, 0].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['H_z_func'](like.cosmo.cosmo_dic['z_win']))
axs[0, 0].set_xlabel(r'$z$')
axs[0, 0].set_ylabel(r'$H(z) / \left( \frac{km}{s\times Mpc}\right)$')
axs[0, 1].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['d_z_func'](like.cosmo.cosmo_dic['z_win']),
              label = r'$D_A(z) / (Mpc)$')
axs[0, 1].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['r_z_func'](like.cosmo.cosmo_dic['z_win']), '--',
              label = r'$r(z)$')
axs[0, 1].set_xlabel(r'$z$')
axs[0, 1].legend()
axs[1, 0].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['sigma8_z_func'](like.cosmo.cosmo_dic['z_win']),
              label = r'$\sigma_8(z)$')
axs[1, 0].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['fsigma8_z_func'](like.cosmo.cosmo_dic['z_win']),
              '--', label = r'$f\sigma_8(z)$')
axs[1, 0].set_xlabel(r'$z$')
axs[1, 0].legend()
axs[1, 1].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['D_z_k'],
              label = r'$D(z)$')
axs[1, 1].plot(like.cosmo.cosmo_dic['z_win'], 
               like.cosmo.cosmo_dic['f_z'](like.cosmo.cosmo_dic['z_win']),
              '--', label = r'$f = f\sigma_8/\sigma_8$')
axs[1, 1].set_xlabel(r'$z$');
axs[1, 1].legend()
plt.subplots_adjust(top=0.90)

In [ ]:
# Let's plot the matter power spectrum and other spectra quantities
# 'Pgg_spec', 'Pgg_phot', 'Pgdelta_phot', 'Pgdelta_spec', 'Pii', 'Pdeltai', 'Pgi_phot', 'Pgi_spec' 
# See http://pf-ist-likelihood.pages.euclid-sgs.uk/likelihood-implementation/likelihood.cosmo.cosmology.html
# for extra information

import warnings
warnings.filterwarnings('ignore')
fig, axs = plt.subplots(9, 2, figsize=(20,44))
fig.suptitle('Spectra')
ks=np.logspace(-5, 0, 100)
axs[0, 0].loglog(ks, like.cosmo.cosmo_dic['Pk_delta'].P(like.cosmo.cosmo_dic['z_win'][0], ks), 'o', 
           label=r"z = {}".format(like.cosmo.cosmo_dic['z_win'][0]))
axs[0, 0].set_xlabel(r'$k$')
axs[0, 0].set_ylabel(r'$P_m / (Mpc^{-3})$')
axs[0, 0].legend()
#------
axs[0, 1].loglog(ks, like.cosmo.cosmo_dic['Pk_delta'].P(like.cosmo.cosmo_dic['z_win'][-1], ks), 'o', 
           label=r"z = {}".format(like.cosmo.cosmo_dic['z_win'][-1]))
axs[0, 1].set_xlabel(r'$k$')
axs[0, 1].set_ylabel(r'$P_m / (Mpc^{-3})$')
axs[0, 1].legend()
#------
for i, k in enumerate(ks):
    axs[1, 0].loglog(k, like.cosmo.cosmo_dic['Pgg_spec'](
    like.cosmo.cosmo_dic['z_win'][0], k, 1), 'ko', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][0]) if i == 0 else "")
    axs[1, 1].loglog(k, like.cosmo.cosmo_dic['Pgg_spec'](
    like.cosmo.cosmo_dic['z_win'][-1], k, 1), 'ko', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][-1]) if i == 0 else "")
axs[1, 0].set_xlabel(r'$k$')
axs[1, 0].set_ylabel(r'$P_{gg}^{spec} / (Mpc^{-3})$')
axs[1, 1].set_xlabel(r'$k$')
axs[1, 1].set_ylabel(r'$P_{gg}^{spec} / (Mpc^{-3})$')
axs[1, 0].legend()
axs[1, 1].legend()

#------
for i, k in enumerate(ks):
    axs[2, 0].loglog(k, like.cosmo.cosmo_dic['Pgg_phot'](
    like.cosmo.cosmo_dic['z_win'][0], k), 'go', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][0]) if i == 0 else "")
    axs[2, 1].loglog(k, like.cosmo.cosmo_dic['Pgg_phot'](
    like.cosmo.cosmo_dic['z_win'][-1], k), 'go', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][-1]) if i == 0 else "")
axs[2, 0].set_xlabel(r'$k$')
axs[2, 0].set_ylabel(r'$P_{gg}^{phot} / (Mpc^{-3})$')
axs[2, 1].set_xlabel(r'$k$')
axs[2, 1].set_ylabel(r'$P_{gg}^{phot} / (Mpc^{-3})$')
axs[2, 0].legend()
axs[2, 1].legend()

#------
for i, k in enumerate(ks):
    axs[3, 0].loglog(k, like.cosmo.cosmo_dic['Pgdelta_spec'](
    like.cosmo.cosmo_dic['z_win'][0], k, 1), 'ko', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][0]) if i == 0 else "")
    axs[3, 1].loglog(k, like.cosmo.cosmo_dic['Pgdelta_spec'](
    like.cosmo.cosmo_dic['z_win'][-1], k, 1), 'ko', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][-1]) if i == 0 else "")
axs[3, 0].set_xlabel(r'$k$')
axs[3, 0].set_ylabel(r'$P_{g\delta}^{spec} / (Mpc^{-3})$')
axs[3, 1].set_xlabel(r'$k$')
axs[3, 1].set_ylabel(r'$P_{g\delta}^{spec} / (Mpc^{-3})$')
axs[3, 0].legend()
axs[3, 1].legend()


#------
for i, k in enumerate(ks):
    axs[4, 0].loglog(k, like.cosmo.cosmo_dic['Pgdelta_phot'](
    like.cosmo.cosmo_dic['z_win'][0], k), 'go', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][0]) if i == 0 else "")
    axs[4, 1].loglog(k, like.cosmo.cosmo_dic['Pgdelta_phot'](
    like.cosmo.cosmo_dic['z_win'][-1], k), 'go', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][-1]) if i == 0 else "")
axs[4, 0].set_xlabel(r'$k$')
axs[4, 0].set_ylabel(r'$P_{g\delta}^{phot} / (Mpc^{-3})$')
axs[4, 1].set_xlabel(r'$k$')
axs[4, 1].set_ylabel(r'$P_{g\delta}^{phot} / (Mpc^{-3})$')
axs[4, 0].legend()
axs[4, 1].legend()

#------
for i, k in enumerate(ks):
    axs[5, 0].loglog(k, like.cosmo.cosmo_dic['Pii'](
    like.cosmo.cosmo_dic['z_win'][0], k), 'ro', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][0]) if i == 0 else "")
    axs[5, 1].loglog(k, like.cosmo.cosmo_dic['Pii'](
    like.cosmo.cosmo_dic['z_win'][-1], k), 'ro', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][-1]) if i == 0 else "")
axs[5, 0].set_xlabel(r'$k$')
axs[5, 0].set_ylabel(r'$P_{ii} / (Mpc^{-3})$')
axs[5, 1].set_xlabel(r'$k$')
axs[5, 1].set_ylabel(r'$P_{ii} / (Mpc^{-3})$')
axs[5, 0].legend()
axs[5, 1].legend()

#------
for i, k in enumerate(ks):
    axs[6, 0].plot(k, like.cosmo.cosmo_dic['Pdeltai'](
    like.cosmo.cosmo_dic['z_win'][0], k), 'bo', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][0]) if i == 0 else "")
    axs[6, 1].plot(k, like.cosmo.cosmo_dic['Pdeltai'](
    like.cosmo.cosmo_dic['z_win'][-1], k), 'bo', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][-1]) if i == 0 else "")
axs[6, 0].set_xlabel(r'$k$')
axs[6, 0].set_ylabel(r'$P_{\delta i} / (Mpc^{-3})$')
axs[6, 1].set_xlabel(r'$k$')
axs[6, 1].set_ylabel(r'$P_{\delta i} / (Mpc^{-3})$')
axs[6, 0].legend(loc=4)
axs[6, 1].legend(loc=4)

#------
for i, k in enumerate(ks):
    axs[7, 0].plot(k, like.cosmo.cosmo_dic['Pgi_phot'](
    like.cosmo.cosmo_dic['z_win'][0], k), 'go', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][0]) if i == 0 else "")
    axs[7, 1].plot(k, like.cosmo.cosmo_dic['Pgi_phot'](
    like.cosmo.cosmo_dic['z_win'][-1], k), 'go', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][-1]) if i == 0 else "")
axs[7, 0].set_xlabel(r'$k$')
axs[7, 0].set_ylabel(r'$P_{gi}^{phot} / (Mpc^{-3})$')
axs[7, 1].set_xlabel(r'$k$')
axs[7, 1].set_ylabel(r'$P_{gi}^{phot} / (Mpc^{-3})$')
axs[7, 0].legend(loc=4)
axs[7, 1].legend(loc=4)

#------
for i, k in enumerate(ks):
    axs[8, 0].plot(k, like.cosmo.cosmo_dic['Pgi_spec'](
    like.cosmo.cosmo_dic['z_win'][0], k), 'ko', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][0]) if i == 0 else "")
    axs[8, 1].plot(k, like.cosmo.cosmo_dic['Pgi_spec'](
    like.cosmo.cosmo_dic['z_win'][-1], k), 'ko', label=r"z = {}".format(
        like.cosmo.cosmo_dic['z_win'][-1]) if i == 0 else "")
axs[8, 0].set_xlabel(r'$k$')
axs[8, 0].set_ylabel(r'$P_{gi}^{spec} / (Mpc^{-3})$')
axs[8, 1].set_xlabel(r'$k$')
axs[8, 1].set_ylabel(r'$P_{gi}^{spec} / (Mpc^{-3})$')
axs[8, 0].legend(loc=4)
axs[8, 1].legend(loc=4)

plt.subplots_adjust(top=0.97)


In [ ]:
# We can also plot the initial data read by the like_calc module (class Euclike). 
# This data is at the moment the fiducial data store within the data folder of the repository

# DISCLAIMER: the data is read in the reader module, but it is saved as attribute of the Euclike class too.
# The instance of the Euclike class is saved as an attribute of the EuclidLikelihood class as well.

# i.e: let's plot the galaxy distributions for GC-photo(z) and WL (AT THE MOMENT, THEY ARE THE SAME)
zs = np.linspace(0, 4, 1000)
fig, axs = plt.subplots(2, 1, figsize=(18,17))
for key, value in like.likefinal.data_ins.nz_dict_GC_Phot.items():
    axs[0].plot(zs, value(zs), label = key)
for key, value in like.likefinal.data_ins.nz_dict_WL.items():
    axs[1].plot(zs, value(zs), label = key)
axs[0].set_xlabel(r'$z$')
axs[0].set_ylabel(r'$n(z)$')
axs[0].set_title(r'$GC-photo(z)$')
axs[0].legend(ncol=2)
axs[1].set_xlabel(r'$z$')
axs[1].set_ylabel(r'$n(z)$')
axs[1].set_title(r'$WL$')
axs[1].legend(ncol=2);
#plt.subplots_adjust(top=0.90)

In [ ]:
# We can also plot the window functions for the photometric observables
# For that, you need to import the photo class and read the cosmology dictionary and the n(z) distributions above
from likelihood.photometric_survey.photo import Photo
photo = Photo(like.cosmo.cosmo_dic, like.likefinal.data_ins.nz_dict_WL, like.likefinal.data_ins.nz_dict_GC_Phot)

In [ ]:
import matplotlib.colors as mcolors
color = list(mcolors.TABLEAU_COLORS.values())
fig, axs = plt.subplots(3, 1, figsize=(18,12))
print('ATTENTION: WL kernel computation takes some time')
# DISCLAIMER: photo.GC_window accepts an array for z, but WL_window does not. 
# This is why we need to go through a foor-loop for WL
for i in range(0, 10):
    print('bin: {}'.format(i+1))
    axs[0].plot(zs, photo.GC_window(zs, i+1), '-', label='n{}'.format(i+1))
    for j, z in enumerate(zs):
        axs[1].plot(z, photo.WL_window(z, i+1), '.', color = color[i], label='n{}'.format(i+1) if j == 0 else "")
    axs[2].plot(zs, photo.IA_window(zs, i+1), label='n{}'.format(i+1))
axs[0].set_xlabel(r'$z$')
axs[0].set_ylabel(r'$W_i^{GC-photo}$')
axs[0].legend(ncol=2)
axs[1].set_xlabel(r'$z$')
axs[1].set_ylabel(r'$W_i^{\gamma}$')
axs[1].legend(ncol=2);
axs[2].set_xlabel(r'$z$')
axs[2].set_ylabel(r'$W_i^{IA}$')
axs[2].legend(ncol=2);

### Plot Euclid final observables

We can use the auxiliary module of the Euclid-box to plot the final observables

In [ ]:
# We import the Plotter Class
from likelihood.auxiliary.plotter import Plotter

In [ ]:
# Weak-Lensing angular power spectrum according to the IST:L recipe 

fig1 = plt.figure()
ax1 = fig1.add_subplot(1, 1, 1)

pl_inst = Plotter(cosmo_dic=like.cosmo.cosmo_dic)

ax1 = pl_inst.plot_external_Cl_phot(1, 1, ax1, probe='WL')
ax1 = pl_inst.plot_Cl_phot(np.logspace(1, 3.6, 10), 1, 1, ax1, probe='WL', pl_colour='r', pl_linestyle='--')
ax1.set_xlabel(r'$\ell$', fontsize=16)
ax1.set_ylabel(r'$C_\ell$', fontsize=16)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_title('Weak-Lensing')
ax1.legend()

In [ ]:
# GC-photo(z) angular power spectrum

fig1 = plt.figure()
ax1 = fig1.add_subplot(1, 1, 1)

ax1 = pl_inst.plot_external_Cl_phot(1, 1, ax1, probe='GC-Phot')
ax1 = pl_inst.plot_Cl_phot(np.logspace(1, 3.6, 10), 1, 1, ax1, probe='GC-Phot', pl_colour='r', pl_linestyle='--')
ax1.set_xlabel(r'$\ell$', fontsize=16)
ax1.set_ylabel(r'$C_\ell$', fontsize=16)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_title('GC-photo(z)')
ax1.legend()

In [ ]:
# Cross-correlation WL x GC-photo(z) angular power spectrum

fig1 = plt.figure()
ax1 = fig1.add_subplot(1, 1, 1)

ax1 = pl_inst.plot_external_Cl_XC(1, 1, ax1)
ax1 = pl_inst.plot_Cl_XC(np.logspace(1, 3.6, 10), 1, 1, ax1, pl_colour='r', pl_linestyle='--')
ax1.set_xlabel(r'$\ell$', fontsize=16)
ax1.set_ylabel(r'$C_\ell$', fontsize=16)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_title('WL x GC-photo(z)')
ax1.legend()

In [ ]:
# GC-spec observable
# Note: The internal and external spectra are currently plotted seperately 
# due to the fact that the external spectra are supplied in different units. 
## Once this is corrected, this notebook should be editted to place both on the same plot.

fig1 = plt.figure()
ax1 = fig1.add_subplot(1, 1, 1)

fig2 = plt.figure()
ax2 = fig2.add_subplot(1, 1, 1)

ax1 = pl_inst.plot_external_GC_spec("1.2", 2, ax1, pl_label='a')
ax1.set_xlabel(r'$k$', fontsize=16)
ax1.set_ylabel(r'$P_\ell$', fontsize=16)
ax1.set_xscale('log')
ax1.set_yscale('log')

ax2 = pl_inst.plot_GC_spec_multipole(1.2, np.linspace(0.01, 0.5), 2, ax2, pl_colour='r', pl_linestyle='--')
ax2.set_xlabel(r'$k$', fontsize=16)
ax2.set_ylabel(r'$P_\ell$', fontsize=16)
ax2.set_xscale('log')
ax2.set_yscale('log')